In [1]:
# Import Libraries

import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import json
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType
import pyspark.sql.functions as F
from kafka import KafkaConsumer

In [2]:
# Configuration

KAFKA_HOSTS = 'localhost:9092'
KAFKA_VERSION = (0, 10)
topics = "male,female,eighteen,twentyseven,forty,sixty,senior,words"

In [7]:
# Spark

spark = SparkSession.builder.master("local[*]") \
    .appName("Profile Stream Consumer") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1") \
    .getOrCreate()

In [8]:
# Read DF

df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_HOSTS) \
  .option("subscribe", topics) \
  .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

DataFrame[key: string, value: string]

In [10]:
# Write the received data to files

for i in topics.split(','):
    sdf = df.filter(df.topic==i)
    count = sdf.count()
    print("Count : ",i, " = ", count)
    if count != 0:
        filename = ('output/'+i+'.csv')
        sdf.select("value").write.format('com.databricks.spark.csv').mode('overwrite').save(filename)

Count :  male  =  354019
Count :  female  =  340519
Count :  eighteen  =  20421
Count :  twentyseven  =  119071
Count :  forty  =  221900
Count :  sixty  =  198368
Count :  senior  =  156366
Count :  words  =  4730918


In [11]:
spark.stop()